# Elastic Index
golden_retriever, inference 시에 매번 새로 동작하는 Elastic Index 생성  
  
이를 효율적으로 하기 위한 방법 실험  
  
Reference : [Adding a FAISS or Elastic Search index to a Dataset](https://huggingface.co/docs/datasets/faiss_and_ea.html)

## Elastic index

In [ ]:
## Elasticsearch start
!service elasticsearch start

 * Starting Elasticsearch Server
 * Already running.
   ...done.


In [ ]:
## Load Libraries
import tqdm
import numpy as np
import pandas as pd
from preprocess import preprocess_retrieval
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch

In [ ]:
## Load augmented data
train = load_from_disk("../data/gen_wiki")["train"]

In [ ]:
# Load wikipedia_documents.json
with open("../data/wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki = json.load(f)
org_contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))
contexts = [preprocess_retrieval(corpus) for corpus in contexts]

In [ ]:
# Elastic Search index training
INDEX_NAME = "wiki_index"

INDEX_SETTINGS = {"settings" : {"index":{"analysis":{"analyzer":{"korean":{"type":"custom",
                                        "tokenizer":"nori_tokenizer","filter": [ "shingle" ],}}}}},
"mappings": {"properties" : {"context" : {"type" : "text","analyzer": "korean","search_analyzer": "korean"},}}}

DOCS = {}
for i in tqdm(range(len(contexts))):
    DOCS[i] = {'context': contexts[i]}
    
try:
    es.transport.close()
except:
    pass

es = Elasticsearch(timeout=30, max_retries=10, retry_in_timeout=True) 

if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

for doc_id, doc in tqdm(DOCS.items(), desc="ES training..!"):
    es.index(index=INDEX_NAME,  id=doc_id, body=doc)

/tmp/ipykernel_23522/598414479.py:18: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
/tmp/ipykernel_23522/598414479.py:20: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=IN

/tmp/ipykernel_23522/598414479.py:23: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=INDEX_NAME,  id=doc_id, body=doc)


In [ ]:
## 원래 golden_retriever 내 함수
def get_relevent_elasticsearch(query, k=10):
    mod_query = preprocess_retrieval(query)
    try:
        res = es.search(index="wiki_index", q=mod_query, size=k)
    except:
        mod_q = mod_query.replace("%", " ").replace("-", " "))
        res = es.search(index="wiki_index", q=mod_q, size=k)
    
    doc_scores = [float(res['hits']['hits'][idx]['_score']) for idx in range(k)]
    doc_indices = [int(res['hits']['hits'][idx]['_id']) for idx in range(k)]
    return doc_scores, doc_indices

In [ ]:
## wiki_index 있는지 확인!
es = Elasticsearch('localhost:9200')
INDEX_NAME = "wiki_index"
es.indices.exists(INDEX_NAME)

/tmp/ipykernel_23522/1910961920.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists(INDEX_NAME)


True

In [ ]:
# 원 방법이랑 똑같은 context 내에서 index 생성
DOCS = {}
for i in tqdm(range(len(org_contexts)), desc="preparing documents"):
    DOCS[i] = {'context': org_contexts[i]}

df_wiki = pd.DataFrame.from_dict(DOCS, orient='index')
df_wiki["preprocessed_text"] = [preprocess_retrieval(corpus) for corpus in df_wiki["context"]]
wiki_dataset = Dataset.from_pandas(df_wiki)  # Convert dict into Dataset
# wiki_dataset.add_elasticsearch_index("preprocessed_text", es_index_name="wiki_index1", es_index_config=INDEX_SETTINGS)

# port에 이미 생성된 elastic search index 가 있을 경우 -> 그를 Dataset에 add
wiki_dataset.add_elasticsearch_index("preprocessed_text", host="localhost", port="9200")

In [ ]:
## Dataset Index Test
idx = 100
print(train["question"][idx])
print(train["context"][idx])
query = train["question"][idx]
scores, retrieved_examples = train.get_nearest_examples("context", query, k=10)
retrieved_examples["context"]

##! Dataset Index로 뽑힌 passage와 gen_relevent elastic search로 탐색된 passage 비교   
    -> 다름! 왜?  
    둘 다 아예 말이 안 되는 건 아닌데..! ; 실제 코드에서 돌려봤을 때 topk 성능도 원래와 비슷

In [ ]:
idx = 0
print(train["question"][idx], train["answers"][idx])
print(train["context"][idx])

query = preprocess_retrieval(train["question"][idx])

scores, indices = wiki_dataset.search("preprocessed_text", query, k=3)
print(scores, indices)
print(retrieved_examples["text"])
print("\n".join([wiki_dataset["preprocessed_text"][idx] for idx in indices[:]]))

doc_scores, doc_indices = get_relevent_elasticsearch(query, k=3)
print(doc_scores, doc_indices)
print("\n".join([preprocess_retrieval(org_contexts[idx]) for idx in doc_indices[:]]))

백장 한국 최후의 칸은 누구인가? 토크타미시 칸
토크타미시 칸(? ~ 1406년)은 백장 한국의 마지막 칸이었다. 그는 백장 한국과 청장 한국을 통합하여 킵차크 한국을 하나의 나라로 만들었다. 그는 징기스칸의 손자 오르다 칸의 후손이다.

토크타미시는 1376년 역사에 등장한다. 그는 백장 한국의 지배자였던 그의 삼촌 우루스를 전복하려고 하였다. 그리고 티무르 황제에게로 달아났다. 1378년 토크타미시는 우루스 사후에도 생존하였고 티무르의 지원으로 백장한국의 제위에 올랐다.

토크타미시는 그의 조상처럼 하기를 꿈꾸었다. 그리고 주치 울루스를 재통일하는 계획을 세웠다. 1380년 그는 청장 한국을 침입하였다. 청장 한국의 군주 마마이는 쿨리코보 전투 직후에 피살되었다.

청장 한국과 백장 한국을 킵차크 한국에 통합한 후에 그는 1382년 모스크바 원정을 성공적으로 이끌었다. 6년 만에 그는 크리미아에서 발하슈 호까지의 킵차크 칸국을 재통일하였다.

1395년에 티무르가 킵차크 한국을 공격하였고 그를 테레크에서 격파하였다. 티무르는 수도 사라이 베르케를 약탈하였고 킵차크 한국을 속국으로 만들었으며 꼭두각시 군주를 왕좌에 세웠다.
[6.9865046, 6.867181, 6.826168] [10278, 31520, 39666]
고시 통화는 다음과 같다 괄호 안에 표시된 숫자는 환전이 가능한 화폐 단위를 의미한다 * 아시아 ** 홍콩 달러 모든 지폐 및 동전 ** 일본 엔 모든 지폐 및 동전 ** 중국 위안 1 2 5 10 20 50 100 ** 태국 밧 20 50 100 500 1 000 ** 싱가포르 달러 2 5 10 20 50 100 1 000 10 000 ** 베트남 동 2 000 5 000 10 000 20 000 50 000 100 000 200 000 500 000 ** 이름 타이완 신 타이완 달러 100 200 500 1 000 2 000 ** 인도네시아 루피아 100 500 1 000 2 000 5 000 10 000 20 000 50 000 100 0